In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import os
from VAERSFileDownloader import updateVAERSFiles
from datetime import datetime
from DateProvider import DateProvider
from InternationalVaersCovid19Provider import getInternationalVaersCovid19, get_international_VAERSVAX_VAERSSYMPTOMS_Covid19
from BatchCodeTableHtmlUpdater import updateBatchCodeTableHtmlFile
from BatchCodeTablePersister import createAndSaveBatchCodeTables
from SymptomByBatchcodeTableFactory import SymptomByBatchcodeTableFactory
from HistogramFactoryAndPersister import createAndSaveGlobalHistograms, createAndSaveHistogramsForCountries
from BatchCodeTableFactory import BatchCodeTableFactory
from BatchCodeTableHavingGuessedCountriesFactory import BatchCodeTableHavingGuessedCountriesFactory


In [ ]:
print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
years_from_start_of_COVID_vaccination_to_present = list(range(2020, datetime.now().year + 1))

In [ ]:
updateVAERSFiles(
        years = years_from_start_of_COVID_vaccination_to_present,
        workingDirectory = os.getcwd())

In [ ]:
international_VAERSVAX_Covid19, international_VAERSSYMPTOMS = get_international_VAERSVAX_VAERSSYMPTOMS_Covid19(years = years_from_start_of_COVID_vaccination_to_present)

In [ ]:
international_VAERSVAX_Covid19

In [ ]:
international_VAERSSYMPTOMS

In [ ]:
symptomByBatchcodeTable = SymptomByBatchcodeTableFactory.createSymptomByBatchcodeTable(international_VAERSVAX_Covid19, international_VAERSSYMPTOMS)
symptomByBatchcodeTable

In [ ]:
createAndSaveGlobalHistograms(symptomByBatchcodeTable)

In [ ]:
createAndSaveHistogramsForCountries(symptomByBatchcodeTable)

In [ ]:
internationalVaersCovid19 = getInternationalVaersCovid19(dataDir = 'VAERS', years = years_from_start_of_COVID_vaccination_to_present)
internationalVaersCovid19

In [ ]:
updateBatchCodeTableHtmlFile(internationalVaersCovid19, batchCodeTableHtmlFile="../docs/batchCodeTable.html")

In [ ]:
createAndSaveBatchCodeTables(
    internationalVaersCovid19,
    minADRsForLethality = 100,
    batchCodeTableFactory = BatchCodeTableHavingGuessedCountriesFactory(BatchCodeTableFactory(internationalVaersCovid19)),
    onCountryProcessed = display)

In [6]:
import pandas as pd
from CountriesByBatchcodeProvider import getCountriesByClickedBatchcode, getCountriesByBatchcodeBeforeDeletion

In [2]:
countriesByClickedBatchcode = getCountriesByClickedBatchcode()
countriesByClickedBatchcode

b'Skipping line 9: expected 22 fields, saw 23\n'


,Countries
Clicked Batchcode,
(not set),"{Germany, Poland, United Kingdom, Australia, B..."
FE6208,"{Germany, Poland, United Kingdom, Australia, B..."
039K20A,"{Sweden, United States, Spain, France, Austral..."
EN6201,"{Sweden, United States, Spain, France, Austral..."
FD6840,"{Sweden, United States, Spain, France, Austral..."
...,...
ZTC540693,{United States}
ZU5A21A,{}
ZZ3249,{Hungary}


In [3]:
countriesByBatchcodeBeforeDeletion = getCountriesByBatchcodeBeforeDeletion()
countriesByBatchcodeBeforeDeletion

,Countries
VAX_LOT,
6362737,[United States]
SCH2,[United States]
NO72A,[Philippines]
EW096,[United States]
FS1926,[United States]
...,...
211221A,[United States]
2112D21A,[United States]
2114214,[United States]


In [41]:
res = pd.merge(
    countriesByClickedBatchcode,
    countriesByBatchcodeBeforeDeletion,
    how = 'inner',
    left_index = True,
    right_index = True,
    suffixes=(' Clicked', ' Before Deletion'))
res.index.name = 'VAX_LOT'
res

,Countries Clicked,Countries Before Deletion
VAX_LOT,,
FE6208,"{Germany, Poland, United Kingdom, Australia, B...","[Austria, Croatia, Czechia, Germany, Greece, H..."
039K20A,"{Sweden, United States, Spain, France, Austral...",[United States]
EN6201,"{Sweden, United States, Spain, France, Austral...","[United Kingdom, United States]"
FD6840,"{Sweden, United States, Spain, France, Austral...","[Austria, Croatia, Cyprus, Czechia, Germany, G..."
FF0680,"{Sweden, United States, Canada, Brazil, German...","[Austria, Croatia, Czechia, Finland, France, G..."
...,...,...
ZTC540693,{United States},[United States]
ZU5A21A,{},[United States]
ZZ3249,{Hungary},[United States]


In [42]:
res['Countries Before Deletion'] = res['Countries Before Deletion'].map(set)
res

,Countries Clicked,Countries Before Deletion
VAX_LOT,,
FE6208,"{Germany, Poland, United Kingdom, Australia, B...","{Lithuania, Spain, Czechia, Greece, Germany, A..."
039K20A,"{Sweden, United States, Spain, France, Austral...",{United States}
EN6201,"{Sweden, United States, Spain, France, Austral...","{United Kingdom, United States}"
FD6840,"{Sweden, United States, Spain, France, Austral...","{Lithuania, Cyprus, Czechia, Greece, Germany, ..."
FF0680,"{Sweden, United States, Canada, Brazil, German...","{Portugal, Sweden, Czechia, France, Finland, G..."
...,...,...
ZTC540693,{United States},{United States}
ZU5A21A,{},{United States}
ZZ3249,{Hungary},{United States}


In [43]:
def intersect(series):
    return sorted(set.intersection(*series))

res['Countries Intersection'] = res.apply(intersect, axis = 'columns')
res

,Countries Clicked,Countries Before Deletion,Countries Intersection
VAX_LOT,,,
FE6208,"{Germany, Poland, United Kingdom, Australia, B...","{Lithuania, Spain, Czechia, Greece, Germany, A...","[Austria, Germany, Hungary, Spain, Sweden, Uni..."
039K20A,"{Sweden, United States, Spain, France, Austral...",{United States},[United States]
EN6201,"{Sweden, United States, Spain, France, Austral...","{United Kingdom, United States}","[United Kingdom, United States]"
FD6840,"{Sweden, United States, Spain, France, Austral...","{Lithuania, Cyprus, Czechia, Greece, Germany, ...","[Austria, Germany, Hungary, Sweden, United Kin..."
FF0680,"{Sweden, United States, Canada, Brazil, German...","{Portugal, Sweden, Czechia, France, Finland, G...","[Austria, Germany, Hungary, Italy, Netherlands..."
...,...,...,...
ZTC540693,{United States},{United States},[United States]
ZU5A21A,{},{United States},[]
ZZ3249,{Hungary},{United States},[]


In [46]:
res[res['Countries Intersection'].apply(lambda s: len(s) == 0)]

,Countries Clicked,Countries Before Deletion,Countries Intersection
VAX_LOT,,,
FH3253,"{Australia, Malaysia}","{Austria, Switzerland, Belgium}",[]
FE2090,"{United Kingdom, Australia, Germany}","{Estonia, Spain, France, Denmark, South Africa...",[]
FF0884,"{Australia, Malaysia, Japan}","{Ireland, Denmark}",[]
FG2872,"{Malaysia, Brazil}","{Georgia, United States, Israel}",[]
FG3716,"{Australia, Italy, Malaysia}","{Estonia, Spain, France, Norway, Austria, Denm...",[]
...,...,...,...
YFL8095,{Japan},{United States},[]
Z07AZ1A,{Brazil},{United States},[]
ZU5A21A,{},{United States},[]


In [50]:
res[res['Countries Intersection'].apply(lambda s: len(s) != 0)]

,Countries Clicked,Countries Before Deletion,Countries Intersection
VAX_LOT,,,
FE6208,"{Germany, Poland, United Kingdom, Australia, B...","{Lithuania, Spain, Czechia, Greece, Germany, A...","[Austria, Germany, Hungary, Spain, Sweden, Uni..."
039K20A,"{Sweden, United States, Spain, France, Austral...",{United States},[United States]
EN6201,"{Sweden, United States, Spain, France, Austral...","{United Kingdom, United States}","[United Kingdom, United States]"
FD6840,"{Sweden, United States, Spain, France, Austral...","{Lithuania, Cyprus, Czechia, Greece, Germany, ...","[Austria, Germany, Hungary, Sweden, United Kin..."
FF0680,"{Sweden, United States, Canada, Brazil, German...","{Portugal, Sweden, Czechia, France, Finland, G...","[Austria, Germany, Hungary, Italy, Netherlands..."
...,...,...,...
XXXXXX,{United States},{United States},[United States]
XYZ98765,{Germany},{Germany},[Germany]
Z01AZ1A,{United States},{United States},[United States]


In [44]:
res.to_excel('tmp/res.xlsx')